In [1]:
import ipywidgets as widgets
import pandas as pd
import ixmp
import message_ix
import xlsxwriter
import os
import datetime
import openpyxl
import matplotlib.pyplot as plt
import random

from ipywidgets import Layout
from message_ix import log
from message_ix.utils import make_df
from message_ix.reporting import Reporter, computations
from ixmp.reporting import configure
from IPython.display import display, clear_output
from openpyxl import load_workbook, Workbook

from functions import make_filepath, write_file
from functions import run_baseline, run_model_from_sheet
from functions import process_inputs, save_results

<IPython.core.display.Javascript object>

In [2]:
# Create baseline scenario when app first runs
run_baseline()

INFO:root:unit `MWa` is already defined in the platform instance
INFO:root:unit `tCO2/kWa` is already defined in the platform instance
INFO:root:unit `USD/kW` is already defined in the platform instance
INFO:root:unit `kWa` is already defined in the platform instance


In [9]:
# Create app class

class App: 
    
    slider_width = '100%'
    compare_figsize = (10,7)
    scen_figsize = (5,3)
    cap_max = 100
    
    def __init__(self):
        self._display_container = widgets.Output()
        self._table_container = widgets.Output()
        
        # Text Components
        title = "Electrifying King's Landing"
        scenario_heading = "Scenario-Specific Characteristics"
        comparison_heading = "Comparison Tools"
        descript_text = "Welcome to the King's Landing energy system simulator. This simulator designs a system with minimal discounted total cost given the user input values of demand, allowable emissions, and maximum percentage of wind generation. The costs and emissions given from each run of the simulator, referred to as a 'Scenario', will be displayed for comparison. Additional information for each scenario can be found in an Excel file of the same name." 
        self._title = widgets.HTML(value=f"<h1>{title}</h1>")
        self._scenario_heading = widgets.HTML(value=f"<h2>{scenario_heading}</h2>")
        self._comparison_heading = widgets.HTML(value=f"<h2>{comparison_heading}</h2>")
        self._cost_heading = widgets.HTML(value=f"<h3>{'Costs'}</h3>")
        self._emi_heading = widgets.HTML(value=f"<h3>{'Emissions'}</h3>")
        self._cap_heading = widgets.HTML(value=f"<h3>{'Generator Capacity'}</h3>")
        #self._description = widgets.Label(value=descript_text)
        self._description = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ descript_text +' </p>')
        
        # Interactive Components
        self._name = self._create_name_field()
        self._demand = self._create_demand()
        self._emibound = self._create_emi_bound()
        self._emibtn = self._create_emiboundbutton()
        self._wind = self._create_wind()
        self._displaybtn = self._create_clear_btn()
    
        # Plots
        #self._plots = []
        self._plot_container1 = widgets.Output()
        self._plot_container2 = widgets.Output()
        self._plot_container3 = widgets.Output()
        self._plot_container4 = widgets.Output()
        self._plot_container5 = widgets.Output()
        self._plot_container6 = widgets.Output()
        self._plot_container7 = widgets.Output()
        self._plot_containers = [self._plot_container1, self._plot_container2, self._plot_container3, self._plot_container4, self._plot_container5, self._plot_container6, self._plot_container7]
        
        # Display
        empty = widgets.Label(value='')
        
        name_label = widgets.Label(value='Scenario Name:')
        pop_label = widgets.Label(value='Current and Projected Population:')
        emi_label = widgets.Label(value='Max. Percent of Coal-Only Emissions Permitted:')
        wind_label = widgets.Label(value='Max. Percent of Renewable Energy from Wind:')
        labels = widgets.VBox([name_label, pop_label, emi_label, wind_label])
        inputs = widgets.VBox([self._name, self._demand, self._emibound, self._wind])
        
        buttons = widgets.HBox([self._displaybtn, self._emibtn])
        #controls = widgets.VBox([widgets.HBox([name_label, self._name]), widgets.HBox([pop_label, self._demand]), widgets.HBox([emi_label, self._emibound])])
        controls = widgets.HBox([labels, inputs])
       
        cost_plots = widgets.HBox([self._plot_container1, self._plot_container2, self._plot_container3])
        emi_plots = widgets.HBox([self._plot_container4])
        cap_plots = widgets.HBox([self._plot_container5, self._plot_container6, self._plot_container7])

        head = widgets.VBox([self._title, self._description, empty, controls, empty, buttons, empty])
        #right_bar = widgets.VBox([self._scenario_heading, vert_plots])
        center = widgets.VBox([self._comparison_heading, self._display_container, self._table_container, empty])
        footer = widgets.VBox([self._scenario_heading, 
                               self._cost_heading, cost_plots, 
                               self._emi_heading, emi_plots,
                               self._cap_heading, cap_plots
                              ])

        grid = widgets.AppLayout(header = head, 
                                left_sidebar = None,
                                center = center, 
                                right_sidebar = None, 
                                footer = footer,
                                justify_content ='center',
                                pane_heights = [1,2,3]
                                )
        
        self.container = grid
        
        self._txt_list = []
        self._x = []
        self._y = []
        self._coal_cap = []
        self._wind_cap = []
        self._pv_cap = []
        self._demand_list = []
        self._init_display()
    
    def _create_name_field(self):
        scen_name = widgets.Text(
            #description='Scenario Name:', 
            value = 'Scenario')
        return scen_name
    
    def _create_demand(self):
        demand = widgets.IntRangeSlider(
                #description='Current and Projected Population:', 
                value=[350000,500000],
                min=0, 
                max=1000000,
                step=50000,
                layout=Layout(width=self.slider_width)
        )
        return demand
    
    def _create_wind(self):
        wind_bound = widgets.IntSlider(
            min = 0, 
            max = self.cap_max,
            value=100,
            step = 5,
            layout=Layout(width=self.slider_width)
        )
        return wind_bound
    
    def _create_emi_bound(self):
        emi_bound = widgets.IntSlider(
            min = 0, 
            max = self.cap_max, 
            step = 5,
            #description = 'Percent of Coal-Only Emissions Permitted:',
            layout=Layout(width=self.slider_width)
        )
        return emi_bound
    
    def _create_emiboundbutton(self):
        btn = widgets.Button(description='Run Model', button_style='primary')
        btn.on_click(self._emibtn_eventhandler)
        return btn
    
    def _create_plot(self, sheet, name, count):
        titles = {'CAP': 'Generator Capacity', 'emi': 'CO2 Emissions', 'ACT': 'Generator Activity', 'inv_cost': 'Investment Costs', 'fix_cost': 'Fixed Costs', 'var_cost': 'Variable Costs', 'inv':'Investment Costs', 'tom':'Operation & Maintenance Costs', 'fom':'Fixed Costs', 'vom':'Variable Costs', 'CAP_NEW':'New Capacity'}
        ylabel = {'CAP': 'Capacity [MWa]', 'emi': 'CO2 Emissions [MtCO2]', 'ACT': 'Activity [MWa]', 'inv_cost': 'Cost [USD]', 'fix_cost': 'Cost [USD]', 'var_cost': 'Cost [USD]', 'inv': 'Cost [USD]', 'tom': 'Cost [USD]', 'fom':'Costs [USD]', 'vom': 'Costs [USD]', 'CAP_NEW':'Capacity [MWa]'}
        fig, ax = plt.subplots(figsize=self.scen_figsize)
        nan_filled = sheet.fillna(method='ffill')
        grouped = nan_filled.groupby(['t'])
        new_data = {}
        for key, item in grouped:
            try:
                new_data[key]=grouped.get_group(key)[0].values.tolist()
            except:
                new_data[key]=grouped.get_group(key)['CAP_NEW'].values.tolist()
        index = [310, 320, 330, 340, 350]
        df = pd.DataFrame(data=new_data, index=index)
        df.plot.bar(stacked=True, ax=ax, title=titles[name])
        plt.xlabel('Year')
        plt.ylabel(ylabel[name])
        with self._plot_containers[count]:
            plt.show()
        
    def _emibtn_eventhandler(self, _):
        for plot_con in self._plot_containers:
            plot_con.clear_output(wait=True)
        #self._plots.clear()
        cost, emissions, sheet_dict, scen_name = process_inputs(self)
        count = 0
        for prop in sheet_dict:
            sheet = sheet_dict[prop]
            self._create_plot(sheet, prop, count)
            count = count + 1
        cap_df = sheet_dict['CAP'].fillna(method='ffill')
        coal_list = [round(cap_df.query('t=="coal_ppl" and ya==310').iloc[0,3], 0), round(cap_df.query('t=="coal_ppl" and ya==320').iloc[0,3], 0), round(cap_df.query('t=="coal_ppl" and ya==330').iloc[0,3], 0), round(cap_df.query('t=="coal_ppl" and ya==340').iloc[0,3], 0), round(cap_df.query('t=="coal_ppl" and ya==350').iloc[0,3], 0)]
        wind_list = [round(cap_df.query('t=="wind_ppl" and ya==310').iloc[0,3], 0), round(cap_df.query('t=="wind_ppl" and ya==320').iloc[0,3], 0), round(cap_df.query('t=="wind_ppl" and ya==330').iloc[0,3], 0), round(cap_df.query('t=="wind_ppl" and ya==340').iloc[0,3], 0), round(cap_df.query('t=="wind_ppl" and ya==350').iloc[0,3], 0)]
        pv_list = [round(cap_df.query('t=="pv_ppl" and ya==310').iloc[0,3], 0), round(cap_df.query('t=="pv_ppl" and ya==320').iloc[0,3], 0), round(cap_df.query('t=="pv_ppl" and ya==330').iloc[0,3], 0), round(cap_df.query('t=="pv_ppl" and ya==340').iloc[0,3], 0), round(cap_df.query('t=="pv_ppl" and ya==350').iloc[0,3], 0)]
        self._coal_cap.append(coal_list)
        self._wind_cap.append(wind_list)
        self._pv_cap.append(pv_list)
        self._demand_list.append([self._demand.value[0], self._demand.value[1]])
        #txt = 'Coal: ' + str(self._coal.value) + '\n Wind: ' + str(self._wind.value) + '\n PV: ' + str(self._solar.value)
        #txt = 'Coal: ' + str(coal_cap) + '\n Wind: ' + str(wind_cap) + '\n PV: ' + str(pv_cap)
        txt = scen_name
        self._txt_list.append(txt)
        self._x.append(cost)
        self._y.append(emissions)
        self._init_display()

    def _create_clear_btn(self):
        btn = widgets.Button(description='Clear Comparison', button_style='warning')
        btn.on_click(self._clear_display)
        return btn
    
    def _clear_display(self, _):
        self._display_container.clear_output(wait=True)
        for plot_con in self._plot_containers:
            plot_con.clear_output(wait=True)
        self._table_container.clear_output(wait=True)
        #self._plots.clear()
        self._x.clear()
        self._y.clear()
        self._txt_list.clear()
        self._coal_cap.clear()
        self._wind_cap.clear()
        self._pv_cap.clear()
        self._demand_list.clear()
        print(self._x)
        self._init_display()
        
    def _init_display(self):
        self._display_container.clear_output(wait=True)
        self._table_container.clear_output(wait=True)
        for plot_con in self._plot_containers:
            plot_con.clear_output(wait=True)
        
        with self._display_container:
            fig, ax = plt.subplots(figsize=self.compare_figsize)
            plt.title('Scenario Comparison')
            plt.xlabel('Cost [USD]')
            plt.ylabel('Emissions [MtCO2]')
            ax.scatter(self._x, self._y)
            length = len(self._x)-1
            for i in range(len(self._txt_list)):
                ax.annotate(self._txt_list[i], (self._x[i], self._y[i]))
            if length >= 0:
                ax.scatter(self._x[length], self._y[length], color='r')
            plt.show()
            
        with self._table_container:
            # Show Dataframe
            data = {
                #'Scenario Name': self._txt_list,
                'Population [310-350]': self._demand_list,
                'Cost [USD]': self._x, 
                'Emissions [MtCO2]': self._y, 
                'Coal Capacity By Year [MWa]': self._coal_cap,
                'Wind Capacity By Year [MWa]': self._wind_cap,
                'PV Capacity By Year [MWa]': self._pv_cap
            }
            df = pd.DataFrame(data=data, index=self._txt_list)
            df.style.hide_index()
            if df.empty != True:
                display(df)


In [10]:
# Run app
app = App()
app.container

AppLayout(children=(VBox(children=(HTML(value="<h1>Electrifying King's Landing</h1>"), HTML(value="<style>p{wo…